In [6]:
"""
Write a YAML file based on fetching a material document that
includes keys not currently accounted for by the API repository.
"""

import json
import os
import yaml
from bson.json_util import dumps as bson_dumps
from pymongo import MongoClient

# http://docs.mongodb.org/manual/reference/connection-string/
client = MongoClient(os.environ["MP_DB_CONN"])
materials = client[os.environ["MP_DB_NAME"]]['materials']

doc = json.loads(bson_dumps(materials.find_one({'task_id': 'mp-188'})))

with open("mp-188.yaml", "w") as f:
    yaml.safe_dump(doc, f, default_flow_style=False)